In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('130141-21data2020.csv')
data1 = pd.read_csv('130141-20data2019.csv')

#data.info()

In [3]:
data = data.drop(['casref_od','casref_do'], axis = 1)
data1 = data1.drop(['casref_od','casref_do'], axis = 1)


In [4]:
data = data.dropna(axis = 0)
data1 = data1.dropna(axis=0)
#data.info()

In [5]:
for i in data.columns:
    if data[i].dtypes == 'O':
        print(i)
        print(data[i].value_counts().count())

vuk
8
vuk_text
8
vuzemi_txt
5369


In [6]:
rok   = data['rok'].astype(object)
print('rok  ')
print(rok.value_counts().count())

rok  
1


In [7]:
stapro_kod   = data['stapro_kod'].astype(object)
print('stapro_kod ')
print(stapro_kod.value_counts().count())

stapro_kod 
8


In [8]:
hodnota = data['hodnota'].astype(object)
print('hodnota')
print(hodnota.value_counts().count())

hodnota
3103


In [9]:
vuzemi_cis = data['vuzemi_cis'].astype(object)
print('vuzemi_cis')
print(vuzemi_cis.value_counts().count())

vuzemi_cis
5


In [10]:
vuzemi_kod = data['vuzemi_kod'].astype(object)
print('vuzemi_kod')
print(vuzemi_kod.value_counts().count())

vuzemi_kod
6553


In [11]:
data = data[['idhod','hodnota','vuzemi_cis','vuzemi_kod','rok']]
data1 = data1[['idhod','hodnota','vuzemi_cis','vuzemi_kod','rok']]
print(data.info())
print(data1.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52440 entries, 0 to 52439
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   idhod       52440 non-null  int64
 1   hodnota     52440 non-null  int64
 2   vuzemi_cis  52440 non-null  int64
 3   vuzemi_kod  52440 non-null  int64
 4   rok         52440 non-null  int64
dtypes: int64(5)
memory usage: 2.4 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 52432 entries, 0 to 52431
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   idhod       52432 non-null  int64
 1   hodnota     52432 non-null  int64
 2   vuzemi_cis  52432 non-null  int64
 3   vuzemi_kod  52432 non-null  int64
 4   rok         52432 non-null  int64
dtypes: int64(5)
memory usage: 2.4 MB
None


In [12]:
data = data.append(data1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104872 entries, 0 to 52431
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   idhod       104872 non-null  int64
 1   hodnota     104872 non-null  int64
 2   vuzemi_cis  104872 non-null  int64
 3   vuzemi_kod  104872 non-null  int64
 4   rok         104872 non-null  int64
dtypes: int64(5)
memory usage: 4.8 MB


In [13]:
data = data.sort_values(by=['vuzemi_cis','vuzemi_kod','rok'])

In [14]:
data.head()

,idhod,hodnota,vuzemi_cis,vuzemi_kod,rok
2742,831489323,16,43,500011,2019
11452,831446968,39,43,500011,2019
16337,831510805,37,43,500011,2019
21228,831402733,2,43,500011,2019
27794,831396666,4,43,500011,2019


In [15]:
data = data.rename(columns = {'idhod' : 'id', 'hodnota' : 'population_mean','vuzemi_cis' : 'region','vuzemi_kod' : 'district','rok':'year'})

In [16]:
#data.info()
data.head()

,id,population_mean,region,district,year
2742,831489323,16,43,500011,2019
11452,831446968,39,43,500011,2019
16337,831510805,37,43,500011,2019
21228,831402733,2,43,500011,2019
27794,831396666,4,43,500011,2019


In [17]:
data.to_csv('2nd_cube.csv')

In [20]:
#  pip install rdflib

import rdflib
from rdflib import Graph, URIRef, BNode, Literal

def load_csv_file_as_object(file_path: str):
    import csv
    result = []
    with open(file_path,encoding = "utf8") as stream:
        reader = csv.reader(stream)
        header = next(reader)  # Skip header
        for line in reader:
            result.append({key: value for key, value in zip(header, line)})
    return result

def as_rdf(content):
    result = Graph()
    prefix = "http://bastien_cadiou.org/"
    for record in content:
        resource = URIRef(f"http://bastien_cadiou.org/resource/{record['id']}")
        result.add((resource, URIRef(prefix + "region"), Literal(record["region"])))
        result.add((resource, URIRef(prefix + "district"), Literal(record["district"])))
        result.add((resource, URIRef(prefix + "year"), Literal(record["year"])))
        result.add((resource, URIRef(prefix + "population_mean"), Literal(record["population_mean"])))
    return result

def print_rdf_as_trig(graph: Graph):
    print(graph.serialize(format="turtle", destination = "2nd_cube.ttl"))
    
file_path = "2nd_cube.csv"
data = load_csv_file_as_object(file_path)
rdf = as_rdf(data)
print_rdf_as_trig(rdf)

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].
